In [29]:
!pip3 install ktrain

In [30]:
import os
import string
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import tensorflow as tf
import ktrain
from ktrain import text
from spacy import displacy
from sklearn.model_selection import train_test_split

nlp = spacy.load('en_core_web_lg')

In [31]:
dir(text)

['AnswerExtractor',
 'EnglishTranslator',
 'SimpleQA',
 'TextExtractor',
 'Transformer',
 'TransformerEmbedding',
 'TransformerSummarizer',
 'Translator',
 'ZeroShotClassifier',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'data',
 'eda',
 'entities_from_array',
 'entities_from_conll2003',
 'entities_from_df',
 'entities_from_gmb',
 'entities_from_txt',
 'extract_filenames',
 'filter_by_id',
 'get_topic_model',
 'learner',
 'load_text_files',
 'load_topic_model',
 'models',
 'ner',
 'pickle',
 'predictor',
 'preprocessor',
 'print_sequence_taggers',
 'print_text_classifiers',
 'print_text_regression_models',
 'qa',
 'seqlen_stats',
 'sequence_tagger',
 'shallownlp',
 'summarization',
 'text_classifier',
 'text_regression_model',
 'textextractor',
 'texts_from_array',
 'texts_from_csv',
 'texts_from_df',
 'texts_from_folder',
 'textutils',
 'translation',
 'zsl']

## Read Data

In [32]:
root_folder = r'/kaggle/input/20news-bydate'
sub_folders = os.listdir(root_folder)
len(sub_folders)

2

In [33]:
sub_folders

['20news-bydate-train', '20news-bydate-test']

In [34]:
classes = os.listdir(os.path.join(root_folder, sub_folders[0]))
classes

['soc.religion.christian',
 'talk.politics.misc',
 'rec.motorcycles',
 'talk.politics.mideast',
 'sci.electronics',
 'sci.space',
 'rec.sport.baseball',
 'misc.forsale',
 'talk.politics.guns',
 'sci.med',
 'comp.sys.mac.hardware',
 'rec.sport.hockey',
 'comp.sys.ibm.pc.hardware',
 'talk.religion.misc',
 'sci.crypt',
 'comp.windows.x',
 'comp.os.ms-windows.misc',
 'rec.autos',
 'comp.graphics',
 'alt.atheism']

In [35]:
text.texts_from_folder?

Signature:
text.texts_from_folder(
    datadir,
    classes=None,
    max_features=20000,
    maxlen=400,
    ngram_range=1,
    train_test_names=['train', 'test'],
    preprocess_mode='standard',
    encoding=None,
    lang=None,
    val_pct=0.1,
    random_state=None,
    verbose=1,
)
Docstring:
```
Returns corpus as sequence of word IDs.
Assumes corpus is in the following folder structure:
├── datadir
│   ├── train
│   │   ├── class0       # folder containing documents of class 0
│   │   ├── class1       # folder containing documents of class 1
│   │   ├── class2       # folder containing documents of class 2
│   │   └── classN       # folder containing documents of class N
│   └── test
│       ├── class0       # folder containing documents of class 0
│       ├── class1       # folder containing documents of class 1
│       ├── class2       # folder containing documents of class 2
│       └── classN       # folder containing documents of class N

Each subfolder should contain docume

In [36]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(datadir=root_folder,
                                                                       classes=classes,
                                                                       maxlen=512,
                                                                       train_test_names=sub_folders,
                                                                       preprocess_mode='bert')

detected encoding: utf-8
Decoding with utf-8 failed 1st attempt - using utf-8 with skips
skipped 60 lines (0.0%) due to character decoding errors
skipped 46 lines (0.0%) due to character decoding errors
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [37]:
x_train

[array([[ 101, 2013, 1024, ...,    0,    0,    0],
        [ 101, 2013, 1024, ...,    0,    0,    0],
        [ 101, 2013, 1024, ..., 1010, 3802,  102],
        ...,
        [ 101, 2013, 1024, ...,    0,    0,    0],
        [ 101, 2013, 1024, ..., 7446, 2063,  102],
        [ 101, 2013, 1024, ...,    0,    0,    0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [38]:
print(len(x_train))
print(len(x_train[0]))
print(len(x_train[1]))

2
11314
11314


In [39]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [40]:
print(len(y_train))
print(len(y_train[0]))

11314
20


In [41]:
preproc

In [42]:
dir(preproc)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_transform_y',
 'c',
 'check_trained',
 'get_classes',
 'get_preprocessor',
 'get_tokenizer',
 'is_chinese',
 'is_nospace_lang',
 'lang',
 'max_features',
 'maxlen',
 'migrate_classes',
 'multilabel',
 'ngram_range',
 'preprocess',
 'preprocess_test',
 'preprocess_train',
 'preprocess_train_called',
 'print_seqlen_stats',
 'process_chinese',
 'seqlen_stats',
 'set_classes',
 'set_multilabel',
 'tok',
 'tok_dct',
 'undo',
 'ytransform']

## Model

In [43]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 512


/opt/conda/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [44]:
learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

## Training

In [45]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
1886/1886 [==============================] - 1242s 595ms/step - loss: 0.8840 - accuracy: 0.7377 - val_loss: 0.5611 - val_accuracy: 0.8243


In [50]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
1886/1886 [==============================] - 1117s 592ms/step - loss: 0.2842 - accuracy: 0.9123 - val_loss: 0.5127 - val_accuracy: 0.8449


In [51]:
learner.evaluate((x_train, y_train))

354/354 [==============================] - 257s 726ms/step
              precision    recall  f1-score   support

           0       0.95      0.94      0.94       480
           1       0.96      0.96      0.96       584
           2       0.95      0.95      0.95       591
           3       0.88      0.93      0.91       590
           4       0.97      0.94      0.95       578
           5       0.99      0.97      0.98       593
           6       0.93      0.98      0.95       585
           7       0.98      0.97      0.98       594
           8       0.99      0.99      0.99       598
           9       0.99      0.99      0.99       597
          10       0.99      1.00      0.99       600
          11       0.99      0.99      0.99       595
          12       0.99      0.94      0.96       591
          13       1.00      0.99      1.00       594
          14       1.00      0.99      0.99       593
          15       0.95      0.98      0.97       599
          16       0.9

array([[449,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   9,   0,   2,   2,  18],
       [  0, 560,   7,  10,   0,   3,   3,   0,   0,   1,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   7, 561,  16,   1,   3,   2,   0,   0,   0,   0,   1,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   1,  16, 550,   9,   1,  11,   0,   0,   0,   0,   0,   2,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   2,   1,  21, 542,   0,  10,   0,   0,   1,   0,   0,   1,
          0,   0,   0,   0,   0,   0,   0],
       [  2,   9,   2,   1,   0, 576,   3,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   1,   4,   1,   0, 572,   4,   1,   0,   0,   0,   1,
          1,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   1,   0,   5, 578,   5,   0,   1,   0,   3,
          0,   0,   0,   1,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   2,   4, 590,   0,   0,  

In [52]:
learner.evaluate((x_test, y_test))

236/236 [==============================] - 171s 726ms/step
              precision    recall  f1-score   support

           0       0.70      0.73      0.72       319
           1       0.78      0.83      0.80       389
           2       0.82      0.84      0.83       394
           3       0.69      0.79      0.73       392
           4       0.83      0.85      0.84       385
           5       0.94      0.78      0.85       395
           6       0.86      0.91      0.89       390
           7       0.95      0.87      0.91       396
           8       0.90      0.90      0.90       398
           9       0.97      0.94      0.96       397
          10       0.97      0.98      0.97       399
          11       0.92      0.90      0.91       396
          12       0.84      0.78      0.81       393
          13       0.94      0.92      0.93       396
          14       0.93      0.93      0.93       394
          15       0.84      0.92      0.88       398
          16       0.7

array([[233,   0,   0,   0,   0,   1,   0,   0,   1,   0,   0,   0,   0,
          3,   7,  21,   1,   1,   5,  46],
       [  0, 323,   8,  10,   7,  10,   3,   1,   0,   0,   1,  13,   6,
          3,   1,   2,   0,   1,   0,   0],
       [  0,  18, 332,  31,   4,   6,   1,   0,   1,   0,   0,   0,   0,
          1,   0,   0,   0,   0,   0,   0],
       [  0,   9,  28, 309,  19,   1,  11,   0,   2,   0,   0,   3,   9,
          0,   1,   0,   0,   0,   0,   0],
       [  0,   4,   1,  41, 329,   0,   8,   0,   0,   0,   0,   0,   2,
          0,   0,   0,   0,   0,   0,   0],
       [  0,  40,  31,   3,   5, 308,   4,   0,   0,   0,   0,   2,   0,
          0,   0,   0,   0,   0,   0,   2],
       [  0,   2,   3,  12,   7,   0, 356,   1,   3,   0,   0,   1,   5,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   1,   0,   0,   5,   0,  10, 345,  16,   0,   0,   1,  15,
          0,   1,   0,   0,   0,   1,   1],
       [  2,   1,   0,   0,   0,   0,   5,  10, 357,   1,   0,  

## deployment
https://medium.com/analytics-vidhya/how-to-deploy-your-neural-network-model-using-ktrain-ae255b134c77

## save 

In [63]:
for i in range(len(learner.model.weights)):
    model.weights[i]._handle_name = model.weights[i].name + "_" + str(i)

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/kaggle/working/')

## Load

In [ ]:
import pickle
from tensorflow.keras.models import load_model

# loading preprocess and model file
features = pickle.load(open('/kaggle/working/tf_model.preproc','rb'))
new_model = load_model('/kaggle/working/tf_model.h5')

## download

In [ ]:
!cd /kaggle/working/

In [ ]:
!ls

In [ ]:
from IPython.display import FileLink

FileLink(r'tf_model.h5')

In [ ]:
FileLink(r'tf_model.preproc')